In [8]:
normalizer = Normalizer()
clustered_dong = pd.read_json("./data/군집상권데이터.json")
menu_data = pd.read_json("./data/메뉴_추상설명_재료_토큰화.json")
cafe_menu_data = pd.read_json("./data/메뉴 데이터.json")

In [6]:
import requests
from flask import Flask, request, jsonify
from flask_cors import CORS
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import Normalizer
import torch
import mglearn
from collections import Counter
from kiwipiepy import Kiwi, TypoTransformer, TypoDefinition
import re
import pandas as pd
import json
import numpy as np
import Levenshtein
from kiwipiepy.utils import Stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
pip install Levenshtein

In [10]:
import requests
from flask import Flask, request, jsonify
from flask_cors import CORS
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import Normalizer
import torch
import mglearn
from collections import Counter
from kiwipiepy import Kiwi, TypoTransformer, TypoDefinition
import re
import pandas as pd
import json
import numpy as np
import Levenshtein
from kiwipiepy.utils import Stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


def connect_mysql():
    try:
        mydb = mysql.connector.connect(
            host="project-db-campus.smhrd.com",
            user="campus_24K_LI_p2_3",
            password="smhrd3",
            database="campus_24K_LI_p2_3",
            port=3307
        )
        return mydb
    except Error as e:
        print("데이터베이스 연결 실패")
        return None

# 연결 종료
def close_connection(connection):
    if connection.is_connected():
        connection.close()
        print("MySQL 연결 종료.")

# db의 데이터를 데이터 프레임으로 받아옴
def fetch_data_from_table(table_name):
    try:
        connection = connect_mysql()
        cursor = connection.cursor()
        query = f"SELECT * FROM {table_name}"  # 조회할 테이블 지정
        cursor.execute(query)
        results = cursor.fetchall()  # 모든 데이터를 가져오기

        # 열 이름 가져오기
        column_names = [desc[0] for desc in cursor.description]
        db_df = pd.DataFrame(results, columns=column_names)
        close_connection(connection)
        return db_df

    except Error as e:
        print("데이터 조회 실패:", e)
        close_connection(connection)
        return None

# Levenshtein 거리 기반 유사도 계산
def levenshtein_similarity(str1, str2):
    # Levenshtein 거리 계산
    distance = Levenshtein.distance(str1, str2)
    # 유사도를 0~1 사이 값으로 변환 (1 - 정규화된 거리)
    max_len = max(len(str1), len(str2))
    return 1 - (distance / max_len)

# 입력 메뉴와 가장 유사한 메뉴 찾기
def find_similar_menu(data, input_menu, threshold=0.5):
    # 메뉴 리스트 가져오기
    menu_list = data["menu"].tolist()
    
    # 각 메뉴에 대해 Levenshtein 유사도 계산
    similarities = [levenshtein_similarity(input_menu, menu) for menu in menu_list]
    
    # 유사도가 가장 높은 메뉴 찾기
    max_similarity = max(similarities)
    print(max_similarity)
    most_similar_index = np.argmax(similarities)
    
    # 임계값 비교
    if max_similarity < threshold:
        return "None"
    else:
        return menu_list[most_similar_index]


# 유사 상권 탐색
def yusa_dong(clustered_dong, dong_pk):
    target_dong_index = clustered_dong[clustered_dong["법정동 번호(PK)"]==dong_pk].index[0]

    same_cluster_dong = clustered_dong[clustered_dong["군집"]==clustered_dong.iloc[target_dong_index,-1]]
    same_cluster_dong.reset_index(drop=True, inplace=True)
    target_dong_index = same_cluster_dong[same_cluster_dong["법정동 번호(PK)"]==dong_pk].index[0]
    
    simil_data = same_cluster_dong.iloc[:,3:-1]
    simil_data_scaled = normalizer.fit_transform(simil_data)

    train_tensor = torch.tensor(simil_data_scaled, dtype=torch.float64)
    euclidean_distances = torch.cdist(train_tensor[target_dong_index].unsqueeze(0), train_tensor).squeeze()
    top_idx = np.argsort(euclidean_distances)[1:10]

    return [same_cluster_dong["동 이름"].iloc[int(idx)] for idx in top_idx]



def data_tokenizing(data):
    data["ingred"] = data["ingred"].replace('\'', '', regex=True)
    kiwi = Kiwi()
    for i in range(len(data["info"])):
        data.iloc[i, 1] = kiwi_tokenizer(data["info"][i], tokenizer=kiwi)
    return data


def ingred_sim(data, menu_name, num=10):
    # info_simil 함수로 유사한 메뉴 추출
    info_sim = info_simil(data, menu_name, 50)
    
    # 유사한 메뉴가 없는 경우 처리
    if info_sim.empty or len(info_sim["menu"]) == 0:
        return ["유사한 메뉴가 존재하지 않습니다..."]

    # TF-IDF 변환
    menu_type = [menu.split(" ")[-1]for menu in info_sim["menu"]]
    
    tfidfv = TfidfVectorizer().fit_transform(info_sim["ingred"]+menu_type)

    # 코사인 유사도 계산
    ingred_simil = cosine_similarity(tfidfv[0], tfidfv)[0]

    # 자기 자신 제외하고 유사도 순으로 정렬
    ingred_simil_index = np.argsort(ingred_simil)[::-1][1:num + 1]

    # 유사도가 낮은 경우 처리
    if len(ingred_simil_index) == 0 or ingred_simil[ingred_simil_index[0]] < 0.2:
        return ["유사한 메뉴가 존재하지 않습니다..."]

    # 결과 반환
    return list(info_sim.iloc[ingred_simil_index]["menu"])


def info_simil(data, menu_name, num=10):
    # 메뉴 이름이 유효한지 확인
    if menu_name not in data["menu"].values:
        print("메뉴 이름이 잘못되었습니다.")
        return pd.DataFrame()  # 빈 데이터프레임 반환

    # 선택된 메뉴의 인덱스 가져오기
    pick_idx = data[data["menu"] == menu_name].index[0]

    # TF-IDF 변환
    tfidfv = TfidfVectorizer().fit_transform(data["tokenized_info"])

    # 코사인 유사도 계산
    cosine_simil = cosine_similarity(tfidfv[pick_idx], tfidfv)[0]

    # 유사도를 기준으로 정렬 (내림차순)
    simil_index = np.argsort(cosine_simil)[::-1]

    # 자기 자신 제외하고 상위 num개 선택
    simil_index = simil_index[1:num + 1]

    # 유사도가 일정 수준 이하인 경우 처리
    if len(simil_index) == 0 or cosine_simil[simil_index[0]] < 0.2:
        print("유사한 메뉴가 존재하지 않습니다.")
        return pd.DataFrame()  # 빈 데이터프레임 반환

    # 결과 반환
    return data.iloc[simil_index]


def input_simil(data, input_data, num=10):
    input_token = kiwi_tokenizer(input_data)
    text_vector = np.array(data["menu"] + " " + data["tokenized_info"])
    t_vector = np.insert(text_vector, 0, input_token)

    tfidfv = TfidfVectorizer().fit_transform(t_vector)
    cosine_simil = cosine_similarity(tfidfv[0], tfidfv)[0]

    simil_index = np.flip(np.argsort(cosine_simil[1:]))  # 자기 자신은 제외

    # 유사도 제한
    threshold = 0.15
    if cosine_simil[simil_index[0] + 1] < threshold:
        return ["유사한 메뉴가 존재하지 않습니다..."]
    else:
        top_matches = data.iloc[simil_index[:num], 0]
        return list(top_matches)


def kiwi_tokenizer(sentence, tokenizer=Kiwi(typos="basic")):
    sentence = sentence.replace(" ", "")
    stopwords = Stopwords()
    get_tags = ["XR", "NNG", "VA"]
    sentence = tokenizer.space(sentence)

    kiwi_result = tokenizer.tokenize(
        sentence, normalize_coda=True, stopwords=stopwords)
    token_list = []
    for token in range(len(kiwi_result)):
        if (kiwi_result[token].tag in get_tags):
            token_list.append(kiwi_result[token].form)
    token_str = " ".join(token_list)
    return token_str


# 키워드 추천 분석 결과 전달
def keyword_menu_send_to_spring(menu_data, keyword, recommanded):
    spring_url = "http://localhost:8087/api/receive-keyword-menu"
    
    try:
        menu_info = [f'[{menu_data[menu_data["menu"]==menu]["info"].values[0]}]' for menu in recommanded]
        menu_ingred = [f'[{menu_data[menu_data["menu"]==menu]["ingred"].values[0]}]' for menu in recommanded]
    except:
        menu_info = []
        menu_ingred = []
    
    payload = {
        "status" : "success",
        "keyword": keyword,
        "recommanded": recommanded,
        "menuInfo": menu_info,
        "menuIngred":menu_ingred
    }
    # json 형식으로 전송
    headers = {"Content-Type": "application/json"}
    print(payload)
    try:
        response = requests.post(spring_url, json=payload, headers=headers)
        response.raise_for_status()
        return payload
    except Exception as e:
        # 예외처리
        print(f"스프링 부트 전송 실패: {str(e)}")
        return {"status": "error", "error": str(e)}


# 분석한 유사 상권 정보 전달
def similar_dong_send_to_spring(similar_dongs):
    spring_url = "http://localhost:8087/api/receive-similar-dongs"
    # 스프링에 dong이라는 이름으로 dong_name 데이터 전송
    payload = {"similarDongs": similar_dongs,
               "status": "success"
              }
    # json 형식으로 전송
    headers = {"Content-Type": "application/json"}
    
    try:
        response = requests.post(spring_url, json=payload, headers=headers)
        response.raise_for_status()  # HTTP 오류 있는지 확인 => 예외 발생
        return payload
    except Exception as e:
        # 예외처리
        print(f"스프링 전송 실패: {str(e)}")
        return {"status": "error", "error": str(e)}


# 유사 메뉴 분석 결과 스프링으로 전송
def similar_menu_recommanded(menu_data, similar_menus, menu_name):
    spring_url="http://localhost:8087/api/receive-similar-menus"
    
    try:
        menu_info = [f'[{menu_data[menu_data["menu"]==menu]["info"].values[0]}]' for menu in similar_menus]
        menu_ingred = [f'[{menu_data[menu_data["menu"]==menu]["ingred"].values[0]}]' for menu in similar_menus]
    except:
        menu_info = []
        menu_ingred=[]
    
    payload={"status":"success",
            "similarMenus":similar_menus,
            "menu": menu_name,
             "menuInfo": menu_info,
             "menuIngred":menu_ingred
            }
        
    headers={"Content-Type" : "application/json"}
    print(payload)
    
    try:
        response = requests.post(spring_url, json=payload, headers=headers)
        response.raise_for_status()
        return payload
    except Exception as e:
        print(f"스프링 전송 실패: {str(e)}")
        return {"status" : "error", "error":str(e)}


# 상권 공통 메뉴 분석 결과 스프링으로 전송
def search_sanggueon_gongtong(menu_data, gongtong_menus, sang1, sang2, sang1_ratio, sang2_ratio):
    spring_url="http://localhost:8087/api/receive-sanggueon-gongtong"
    
    try:
        menu_info = [f'[{menu_data[menu_data["menu"]==menu]["info"].values[0]}]' for menu in gongtong_menus if menu in list(menu_data["menu"])]
        menu_ingred = [f'[{menu_data[menu_data["menu"]==menu]["ingred"].values[0]}]' for menu in gongtong_menus if menu in list(menu_data["menu"])]
    except:
        menu_info = []
        menu_ingred = []
    
    payload={"status":"success",
            "gongtongMenus" : gongtong_menus,
             "sang1" : sang1,
             "sang2":sang2,
             "sang1MenuCnt":sang1_ratio,
             "sang2MenuCnt":sang2_ratio,
             "menuInfo":menu_info,
             "menuIngred":menu_ingred
            }
        
    headers={"Content-Type" : "application/json"}
    print(payload)
    
    try:
        response = requests.post(spring_url, json=payload, headers=headers)
        response.raise_for_status()
        return payload
    except Exception as e:
        print(f"스프링 전송 실패: {str(e)}")
        return {"status" : "error", "error":str(e)}


# 상권2만 언급이 많은 메뉴 전송
def search_sanggueon_only(menu_data, only_menus, sang1, sang2, sang1_ratio, sang2_ratio):
    spring_url="http://localhost:8087/api/recieve-sanggueon-only"
    
    try:
        menu_info = [f'[{menu_data[menu_data["menu"]==menu]["info"].values[0]}]' for menu in only_menus if menu in list(menu_data["menu"])]
        menu_ingred = [f'[{menu_data[menu_data["menu"]==menu]["ingred"].values[0]}]' for menu in only_menus if menu in list(menu_data["menu"])]
    except:
        menu_info = []
        menu_ingred = []
    
    payload={"status":"success",
             "onlyMenus":only_menus,
             "sang1":sang1,
             "sang2":sang2,
             "sang1MenuCnt":sang1_ratio,
             "sang2MenuCnt":sang2_ratio,
             "menuInfo":menu_info,
             "menuIngred":menu_ingred
            }
    headers={"Content-Type":"application/json"}
    print(payload)
    
    try:
        response=requests.post(spring_url,json=payload,headers=headers)
        response.raise_for_status()
        return payload
    except Exception as e:
        print(f"스프링 전송 실패: {str(e)}")
        return {"status":"error", "error":str(e)}


# 좌표를 통해 법정동 번호 조회
def get_dong_number(latitude, longitude, dong_data=clustered_dong):
    url = f"https://dapi.kakao.com/v2/local/geo/coord2address.json?x={longitude}&y={latitude}"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}

    response = requests.get(url, headers=headers)

    # 200번 : 통신 성공
    if response.status_code == 200:
        address_info = response.json()
        # 카카오 api에 해당 좌표 정보가 있는지 확인
        if address_info['documents']:
            addr = address_info['documents'][0]['address']['address_name']
            region = addr.split(" ")[0]
            gu = addr.split(" ")[1]
            dong = addr.split(" ")[2]

            region_dong = dong_data[dong_data["지역"].str.contains(region)]
            dong_number = list(
                region_dong[region_dong["동 이름"] == dong].head(1)["법정동 번호(PK)"])[0]
            return dong_number
    return None


# 두 상권 중 다른 상권에만 유독 더 많이 파는 메뉴
def yusasang_only(data, sang1, sang2):
    sang1data = data[data["dong"]==sang1]
    sang2data = data[data["dong"]==sang2]
    
    sang1_count = Counter(sang1data['menu_keyword'])
    sang2_count = Counter(sang2data['menu_keyword'])

    sang1_menu = list(sang1_count.keys())
    sang1_cnt = sang1_count.values()
    sang1_cnt_list = list(sang1_cnt)
    sang1_rat = []
    sang1_sum = 0

    sang2_menu = list(sang2_count.keys())
    sang2_cnt = sang2_count.values()
    sang2_cnt_list = list(sang2_cnt)
    sang2_rat = []
    sang2_sum = 0

    for i in range(len(sang1_cnt)):
        sang1_sum+=sang1_cnt_list[i]
    for i in range(len(sang2_cnt)):
        sang2_sum+=sang2_cnt_list[i]

    sang1_rat = np.array([r/sang1_sum for r in sang1_cnt_list])
    sang2_rat = np.array([r/sang2_sum for r in sang2_cnt_list])

    sang1_most_rat = np.flip(np.argsort(sang1_rat)[:])
    sang2_most_rat = np.flip(np.argsort(sang2_rat)[:])

    sang1_most = [sang1_menu[menu_i] for menu_i in sang1_most_rat]
    sang2_most = [sang2_menu[menu_i] for menu_i in sang2_most_rat]

    only_sang2 = []
    sang1_ratio = []
    sang2_ratio = []
    popularity = len(sang1_most)//5
    
    for i in range(len(sang2_most)):
        if(sang2_most[i] not in sang1_most[:popularity]):
            only_sang2.append(sang2_most[i])
            try:
                sang1_ratio.append(sang1_cnt_list[sang1_menu.index(sang2_most[i])])
            except:
                sang1_ratio.append(0)
            sang2_ratio.append(sang2_cnt_list[sang2_most_rat[i]])
        if(len(only_sang2) == 5):
            break
    if(len(only_sang2) < 3):
        only_sang2.append("두 상권의 거의 모든 메뉴가 유사합니다.")
        sang1_ratio.append(0)
        sang2_ratio.append(0)

    return [only_sang2, sang1_ratio, sang2_ratio]


#두 상권 모두 많이 파는 메뉴
def yusasang_gongtong(data, sang1, sang2):
    sang1data = data[data["dong"]==sang1]
    sang2data = data[data["dong"]==sang2]
    
    sang1_count = Counter(sang1data['menu_keyword'])
    sang2_count = Counter(sang2data['menu_keyword'])

    sang1_menu = list(sang1_count.keys())
    sang1_cnt = sang1_count.values()
    sang1_cnt_list = list(sang1_cnt)
    sang1_rat = []
    sang1_sum = 0

    sang2_menu = list(sang2_count.keys())
    sang2_cnt = sang2_count.values()
    sang2_cnt_list = list(sang2_cnt)
    sang2_rat = []
    sang2_sum = 0

    for i in range(len(sang1_cnt)):
        sang1_sum+=sang1_cnt_list[i]
    for i in range(len(sang2_cnt)):
        sang2_sum+=sang2_cnt_list[i]

    sang1_rat = np.array([r/sang1_sum for r in sang1_cnt_list])
    sang2_rat = np.array([r/sang2_sum for r in sang2_cnt_list])

    sang1_most_rat = np.flip(np.argsort(sang1_rat)[:])
    sang2_most_rat = np.flip(np.argsort(sang2_rat)[:])

    sang1_most = [sang1_menu[menu_i] for menu_i in sang1_most_rat]
    sang2_most = [sang2_menu[menu_i] for menu_i in sang2_most_rat]
    
    sang1_ratio = []
    sang2_ratio = []
    popularity = len(sang2_most) // 10
    
    gongtong_sang = []
    for i in range(popularity, len(sang1_most)):
        if(sang1_most[i] in sang2_most[:popularity*2]):
            gongtong_sang.append(sang1_most[i])
            sang1_ratio.append(sang1_cnt_list[sang1_most_rat[i]])
            sang2_ratio.append(sang2_cnt_list[sang2_menu.index(sang1_most[i])])
        if(len(gongtong_sang)==5):
            break
    if(len(gongtong_sang) < 3):
        gongtong_sang.append("두 상권의 메뉴는 거의 유사하지 않습니다.")
    print(sang1_ratio)
    print(sang2_ratio)
    return [gongtong_sang, sang1_ratio, sang2_ratio]


# 현재 상권에서 주로 파는 메뉴
def sang_menu(data, sang, no_menus=[]):
    sangdata = data[data["dong"]==sang]
    
    sang_count = Counter(sangdata['menu_keyword'])
    sang_most = sang_count.most_common(len(sang_count))
    
    sang_best_menu = []
    for i in range(5, len(sang_most)):
        if(sang_most[i][0] in no_menus):
            continue
        sang_best_menu.append(sang_most[i][0])
        if(len(sang_best_menu)==5):
            break
    if(len(sang_best_menu)<3):
        sang_best_menu.append("해당 상권은 분석할 수 없습니다. 사유 : 상권 내 카페 수 부족")
    return sang_best_menu


app = Flask(__name__)
CORS(app)  # 모든 도메인에서의 요청 허용

# CORS(app, resources={r"/api/*": {"origins": "http://localhost:8087"}})
# REST API key
KAKAO_API_KEY = "015412fbc48c3a4e31b1926b6adb667e"


# 스프링 부트로부터 사용자가 클릭한 좌표 정보를 전달받아 동을 찾은 후 분석하고 전송
@app.route('/send-coordinates', methods=['POST'])
def receive_coordinates():
    # 클라이언트로부터 받은 데이터(현 위치 위도, 경도)
    data = request.get_json()
    latitude = data.get('latitude')
    longitude = data.get('longitude')
    
    url = f"https://dapi.kakao.com/v2/local/geo/coord2address.json?x={longitude}&y={latitude}"
    headers = {"Authorization" : f"KakaoAK {KAKAO_API_KEY}"}
    
    response = requests.get(url, headers=headers)
    
    
    dong_number = get_dong_number(latitude, longitude)
    if(dong_number == None):
        return jsonify({"status" : "error", "message" : "현재 지역의 법정동 번호를 찾을 수 없습니다."})
    
    # 유사 상권의 법정동 번호
    similar_dongs = yusa_dong(clustered_dong, dong_number)
    
    # 스트링 부트로 전달
    spring_response = similar_dong_send_to_spring(similar_dongs)
    # 전송 결과 확인
    if(spring_response.get("status")=="success"):
        return jsonify(spring_response)
    else:
        return jsonify({
            "status" : "error",
            "yusadongs" : "",
            "message" : spring_response.get("error")
        })


# 유사 상권 탐색 2. 스프링으로부터 동을 전달받아 유사한 동 분석
@app.route("/yusa-dong-search", methods=["POST"])
def receive_dong():
    data = request.get_json()
    input_data = data.get("dong").strip()
    dong = "-1"
    region = "-1"
    print(input_data)
    
    yusa_dong_list = []
    
    if(len(input_data.split(" "))>1):
        dong = input_data.split(" ")[-1]
        region = input_data.split(" ")[0]
    else:
        dong = input_data
    
    if(dong not in list(clustered_dong["동 이름"])):
        yusa_dong_list.append("None")
        return jsonify({"status":"success",
                        "dong":"None",
                        "yusaDongNum":"None"
                        })
        
    else:
        # 혹시 지역까지 입력해주면 더 확실하게 동을 찾아 줄 수 있음(동 이름이 겹치는 경우가 많음...)
        if(clustered_dong["지역"].str.contains(region).sum()>0):
            region_data = clustered_dong[clustered_dong["지역"].str.contains(region)]
            dong_data = region_data[region_data["동 이름"]==dong]
        else:
            dong_data = clustered_dong[clustered_dong["동 이름"]==dong]
    
        dong_pk = int(dong_data["법정동 번호(PK)"].values[0])
        yusa_dong_number = yusa_dong(clustered_dong, dong_pk)
        
        payload ={"status":"success",
                 "dong":dong,
                 "yusaDongNum":yusa_dong_number}
        
        return jsonify(payload)
        
        
    

# 스프링 부트로부터 키워드 전달받아 분석
@app.route("/analyze-keyword", methods=["POST"])
def analyze_keyword():
    # 요청 본문 데이터 확인 : 키워드
    data = request.get_json()
    keyword = data.get("keyword", "")
    recommanded = input_simil(menu_data, keyword)

    spring_response = keyword_menu_send_to_spring(menu_data, keyword, recommanded)

    if(spring_response.get("status")=="success"):
        return jsonify(spring_response)
    else:
        return jsonify({"status":"error",
                       "message":spring_response.get("error")})

# 스프링 부트로부터 메뉴 전달받아 분석
@app.route("/analyze-menu", methods=["POST"])
def analyze_menu():
    # 요청 데이터 확인 : 메뉴
    data = request.get_json()
    menu_name = data.get("menu","")
    
    similar_menu_name="None"
    similar_menus = []
    # 사용자 추가 메뉴의 경우 메뉴 데이터에 있을 수도 있고 없을 수도 있음
    # 따라서 메뉴 데이터에 없는 경우를 대비해야 함
    if(menu_name in list(menu_data["menu"])):
        similar_menus = ingred_sim(menu_data, menu_name)
    else:
        similar_menu_name = find_similar_menu(menu_data, menu_name)
        if(similar_menu_name == "None"):
            print("None")
            # 유사한 메뉴 데이터가 없을 경우 키워드 추천 알고리즘으로 추천
            similar_menus=input_simil(menu_data, menu_name)
        else:
            # 유사한 메뉴 데이터가 있는 경우 그 메뉴에 대한 유사 메뉴 추천
            print(similar_menu_name)
            
            similar_menus = ingred_sim(menu_data, similar_menu_name)
    
    spring_response = similar_menu_recommanded(menu_data, similar_menus, menu_name)
    spring_response["menu"]=menu_name
    spring_response["changeMenu"]=similar_menu_name

    if(spring_response.get("status")=="success"):
        return jsonify(spring_response)
    else:
        return jsonify({"status":"error",
                       "message":spring_response.get("error")})

@app.route("/sanggueon-gongtong", methods=["POST"])
def sanggueon_gongtong():
    data = request.get_json()
    sang1 = data.get("sang1", "")
    sang2 = data.get("sang2", "")
    gongtong_menus, sang1_cnt, sang2_cnt = yusasang_gongtong(cafe_menu_data, sang1, sang2)
    print(gongtong_menus)
    spring_response = search_sanggueon_gongtong(menu_data, gongtong_menus,sang1, sang2, sang1_cnt, sang2_cnt)
    if(spring_response.get("status")=="success"):
        return jsonify(spring_response)
    else:
        return jsonify({"status":"error",
                       "message":spring_response.get("error")})

@app.route("/sanggueon-only", methods=["POST"])
def sanggueon_only():
    data=request.get_json()
    sang1=data.get("sang1","")
    sang2=data.get("sang2","")
    only_menus, sang1_cnt, sang2_cnt = yusasang_only(cafe_menu_data, sang1, sang2)
    print(only_menus)
    spring_response = search_sanggueon_only(menu_data, only_menus, sang1, sang2, sang1_cnt, sang2_cnt)
    print(spring_response)
    if(spring_response.get("status")=="success"):
        return jsonify(spring_response)
    else:
        return jsonify({"status":"error","message":spring_response.get("error")})


if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5001)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://192.168.219.78:5001
Press CTRL+C to quit
127.0.0.1 - - [08/Jan/2025 08:49:14] "POST /analyze-menu HTTP/1.1" 200 -


{'status': 'success', 'similarMenus': ['꿀 아메리카노', '헤이즐넛 에스프레소', '헤이즐넛 콜드브루', '얼그레이 아메리카노', '디카페인 아메리카노', '돌체 에스프레소', '바닐라 에스프레소', '커피 프라페', '콜드 브루 딜라이트', '크림 아인슈페너'], 'menu': '플랫화이트', 'menuInfo': ['[진한 맛, 달콤함, 고급스러움, 깔끔함, 자연스러움, 자연스러운 단맛, 진한 커피 풍미, 은은한 꿀 향, 깔끔한 마무리]', '[고소함, 깊은 맛, 진한 커피향, 따뜻함, 고급스러움, 고소한 견과류 향, 진한 커피 맛, 부드러운 끝맛, 풍부한 텍스처, 깊은 여운]', '[고소함, 진한 커피맛, 깔끔함, 부드러움, 고급스러움, 고소한 견과류 풍미, 진한 커피 맛, 시원한 질감, 균형감, 고급스러운 끝맛]', '[허브향, 진한 맛, 깔끔함, 우아함, 고급스러움, 우아한 베르가못 향, 깊은 커피 맛, 부드러운 산미, 깔끔한 마무리, 세련된 풍미]', '[깔끔함, 가벼움, 건강함, 진한 커피맛, 편안함, 가벼운 커피 풍미, 깔끔한 마무리, 온화함, 부담 없는 선택, 은은한 아로마]', '[달콤함, 진한 맛, 크리미함, 부드러움, 고급스러움, 진한 커피 풍미, 크리미한 단맛, 고급스러운 질감, 균형 잡힌 조화, 세련된 마무리]', '[부드러움, 진한 맛, 깔끔함, 고급스러움, 달콤함, 진한 커피 향, 은은한 단맛, 부드러운 여운]', '[진한 커피맛, 크리미함, 부드러움, 만족감, 깔끔함, 깊은 커피 향, 부드러운 크림, 시원한 질감, 달콤한 조화, 만족스러운 맛]', '[진한 커피 풍미, 크리미함, 달콤한 밸런스, 부드러운 텍스처, 고급 음료, 클래식 감성, 만족감, 깔끔함, 농후함, 사랑스러움]', '[크리미함, 진한 맛, 달콤함, 고급스러움, 특별함, 진한 커피 향, 풍성한 크림, 부드러운 끝맛, 고급스러운 질감]'], 'menuIngred': ["[ '에스프레소, 물, 꿀, 얼음']", "[ '에스프레소, 헤이즐

127.0.0.1 - - [08/Jan/2025 08:49:17] "POST /analyze-menu HTTP/1.1" 200 -


0.8333333333333334
레몬 에이드
{'status': 'success', 'similarMenus': ['한라봉 플랫치노', '레몬 블렌디드', '한라봉 에이드', '한라봉 주스', '한라봉 스무디', '한라봉 아이스티', '유자 쉐이크', '요거트 스무디', '애플 아이스티', '라임 소다'], 'menu': '레몬에이드', 'menuInfo': ['[상큼함, 청량감, 과일향, 달콤함, 제주감성, 상큼한 제주 감귤 향, 부드러운 텍스처, 청량한 감미, 자연스러운 단맛, 여운이 남는 풍미]', '[상큼함, 청량감, 새콤달콤, 시원함, 리프레시, 새콤한 시트러스, 청량한 산미, 상쾌함, 자연스러운 단맛, 시원한 질감]', '[상큼함, 과일향, 청량감, 달콤함, 제주감성, 상큼한 감귤향, 제주 감성, 부드러운 단맛, 활력]', '[상큼함, 과일향, 청량감, 달콤함, 제주감성, 상큼한 감귤향, 제주 감성, 달콤한 끝맛, 자연스러운 과즙]', '[상큼함, 과일향, 청량감, 달콤함, 제주감성, 상큼한 감귤 향, 부드러운 텍스처, 제주 감성, 자연스러운 달콤함]', '[상큼함, 청량감, 과일향, 달콤함, 제주감성, 상큼한 감귤 향, 자연스러운 단맛, 여운이 남는 산미, 여름의 감성]', '[상큼함, 부드러움, 달콤함, 과일향, 건강함, 상큼한 감귤 향, 부드러운 크림, 자연스러운 단맛, 청량감, 여름의 생기]', '[건강함, 상큼함, 부드러움, 새콤달콤, 신선함, 상쾌한 요거트, 자연스러운 단맛, 부드러운 텍스처, 활력감, 가벼운 시트러스]', '[상큼함, 청량감, 과일향, 깔끔함, 활기참, 달콤한 사과 향, 상쾌한 텍스처, 자연스러운 감미, 깔끔한 끝맛]', '[새콤달콤, 청량감, 상쾌함, 여름, 깔끔함, 새콤한 시트러스 향, 청량한 탄산, 자연스러운 산미, 경쾌한 감미, 여름의 활력]'], 'menuIngred': ["[ '한라봉', '우유', '얼음', '바닐라 시럽', '휘핑크림']", "[ '레몬', '설탕', '얼음', '우유']", "[ '한라봉

...

0.6666666666666667
크림 파이
{'status': 'success', 'similarMenus': ['치즈 타르트', '밀크 마들렌', '크림 몽블랑', '바닐라 마들렌', '바나나 파이', '고구마 도넛', '와플', '크림 카스테라', '크루키', '카스테라 크루아상'], 'menu': '크림 파스타', 'menuInfo': ['[고소함, 부드러움, 바삭한 타르트지, 크리미한 치즈 필링, 클래식 디저트, 고급스러움, 만족감, 풍미 깊음, 심플함, 깔끔함]', '[부드러움, 달콤함, 촉촉한 질감, 고소한 우유 풍미, 클래식 디저트, 고급스러움, 만족감, 심플함, 화사함, 사랑스러움]', '[부드러움, 달콤함, 크리미한 밤 크림, 촉촉한 질감, 클래식 디저트, 고급스러움, 만족감, 화려함, 정겨움, 풍미 깊음]', '[고소함, 부드러움, 달콤한 바닐라 풍미, 촉촉한 질감, 클래식 디저트, 고급스러움, 만족감, 심플함, 깔끔함, 정겨움]', '[달콤함, 부드러운 바나나 필링, 바삭한 크러스트, 건강한 감성, 고급 디저트, 클래식 감성, 만족감, 촉촉함, 심플함, 사랑스러움]', '[달콤함, 고소함, 바삭한 겉면, 부드러운 고구마 필링, 건강한 디저트, 자연주의, 만족감, 화려함, 사랑스러움, 정겨움]', '[바삭함, 달콤함, 부드러운 속, 풍부한 크림 또는 토핑, 클래식 디저트, 고급스러움, 만족감, 화려함, 촉촉함, 사랑스러움]', '[부드러움, 달콤함, 촉촉한 질감, 크리미한 크림, 클래식 디저트, 고급스러움, 만족감, 화사함, 심플함, 사랑스러움]', '[고소함, 바삭함, 부드러운 크림 필링, 촉촉한 질감, 독특한 조합, 클래식 디저트, 만족감, 고급스러움, 화려함, 풍부함]', '[바삭함, 달콤함, 부드러운 카스테라, 촉촉한 크루아상, 클래식 디저트, 고급스러움, 만족감, 화려함, 독특함, 정겨움]'], 'menuIngred': ["[ '버터', '밀가루', '설탕', '소금', '계란', '크림치즈', '설탕', '바닐라 익스트랙', '휘

127.0.0.1 - - [08/Jan/2025 08:50:31] "POST /analyze-menu HTTP/1.1" 200 -
127.0.0.1 - - [08/Jan/2025 08:50:32] "POST /analyze-menu HTTP/1.1" 200 -


{'status': 'success', 'similarMenus': ['밀크 코코아', '시나몬 모카', '크림 카페모카', '쇼콜라 라떼', '헤이즐넛 쇼콜라', '모카 프라페', '초코 마들렌', '아몬드 모카', '초콜렛 모카', '모카 아인슈페너'], 'menu': '카페모카', 'menuInfo': ['[초콜릿, 달콤함, 부드러움, 따뜻함, 포근함, 진한 초콜릿 풍미, 부드러운 크림, 포근한 달콤함, 고급스러운 여운]', '[스파이시, 초콜릿, 달콤함, 따뜻함, 고급스러움, 따뜻한 계피 향, 진한 초콜릿, 크리미한 질감, 달콤한 조화, 고급스러운 깊이]', '[초콜릿, 크리미함, 달콤함, 진한 맛, 고급스러움, 진한 초콜릿 풍미, 부드러운 크림, 달콤한 조화, 깊고 풍부한 맛, 세련된 만족감]', '[진한 초콜릿, 고급스러움, 달콤함, 풍미, 포근함, 진한 초콜릿 향, 중후함, 감미로운 깊이, 풍부한 크림감, 고급 디저트 느낌]', '[고소함, 초콜릿, 달콤함, 고급스러움, 크리미함, 고소한 견과 향, 진한 초콜릿, 크리미한 질감, 세련된 단맛, 깊은 여운]', '[진한 초콜릿, 달콤함, 크리미함, 풍미, 만족감, 진한 초콜릿 풍미, 크리미한 텍스처, 달콤한 만족감, 고급스러움, 깊은 여운]', '[달콤함, 부드러움, 진한 초콜릿, 촉촉함, 프랑스 감성, 클래식 디저트, 풍미 깊음, 만족감, 고급스러움, 정겨움]', '[고소함, 초콜릿, 달콤함, 크리미함, 풍부함, 고소한 견과 향, 진한 초콜릿, 부드러운 크리미함, 달콤한 만족감, 고급스러운 맛]', '[진한 초콜릿, 풍부함, 달콤함, 따뜻함, 만족감, 달콤한 초콜릿, 진한 크림, 고급스러운 풍미, 풍부한 텍스처, 따뜻한 마무리]', '[진한 초콜릿, 크리미함, 고급스러움, 달콤함, 특별함, 진한 초콜릿과 커피의 조화, 부드러운 크림, 달콤한 끝맛, 고급스러운 풍미, 깊은 만족감]'], 'menuIngred': ["[ '우유', '코코아 파우더', '설탕', '휘핑크림', '초콜릿 시럽']", "[ 

127.0.0.1 - - [08/Jan/2025 08:50:36] "POST /analyze-menu HTTP/1.1" 200 -
127.0.0.1 - - [08/Jan/2025 08:50:36] "POST /analyze-menu HTTP/1.1" 200 -


{'status': 'success', 'similarMenus': ['얼그레이 스무디', '얼그레이 버블티', '바닐라 에스프레소', '우롱 밀크티', '얼그레이 밀크티', '홍차 스무디', '라벤더 밀크티', '홍차 라떼', '샤인머스켓 스무디', '타로 우유'], 'menu': '얼그레이 티', 'menuInfo': ['[허브향, 부드러움, 우아함, 상쾌함, 깔끔함, 우아한 베르가못 향, 부드러운 텍스처, 고급스러운 감미, 청량감, 자연스러운 조화]', '[우아함, 쫄깃함, 허브향, 크리미함, 고급스러움, 우아한 베르가못 향, 쫄깃한 펄, 부드러운 조화, 고급스러운 맛, 균형 잡힌 풍미]', '[부드러움, 진한 맛, 깔끔함, 고급스러움, 달콤함, 진한 커피 향, 은은한 단맛, 부드러운 여운]', '[전통적, 부드러움, 깊은 맛, 고급스러움, 깔끔함, 은은한 허브향, 부드러운 크리미함, 전통적 깊이, 자연스러운 단맛, 고급스러운 풍미]', '[우아함, 부드러움, 깊은 맛, 클래식, 고급스러움, 베르가못 향, 부드러운 크림, 우아한 맛, 조화로움, 깔끔한 끝맛]', '[우아함, 부드러움, 진한 맛, 크리미함, 고급스러움, 깊고 은은한 풍미, 부드러운 텍스처, 고급스러운 맛, 깔끔한 단맛, 세련된 조화]', '[우아함, 허브향, 부드러움, 달콤함, 편안함, 은은한 꽃향, 부드러운 크림, 세련된 풍미, 편안한 끝맛, 우아한 조화]', '[부드러움, 우아함, 진한 맛, 따뜻함, 고급스러움, 은은한 홍차 향, 부드러운 우유, 자연스러운 감미]', '[이국적, 청량감, 과일향, 고급스러움, 달콤함, 고급스러운 과일향, 달콤한 여운, 부드러운 텍스처, 신선한 질감]', '[부드러움, 고소함, 독특함, 달콤함, 크리미함, 부드러운 타로 향, 크리미한 질감, 고소한 단맛, 독특한 풍미, 편안한 여운]'], 'menuIngred': ["[ '얼그레이 티', '우유', '설탕', '얼음', '바닐라 시럽']", "[ '얼그레이 차, 우유, 설탕, 타피오카 펄, 얼음, 바닐라 